<a href="https://colab.research.google.com/github/Colsai/scott_data606/blob/main/LDA_vis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
###########################
# Load CSVs (Github)      #
###########################
!pip install pyLDAvis
!python -m spacy download en_core_web_md -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=5bec61522536db2e8871d4aa466b33b20ecf071b7f59b0c19953328f4b1d6be7
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
     |████████████████████████████████| 33.5 MB 1.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
###########################
# Load CSVs (Github)      #
###########################
import pandas as pd
import sklearn
import nltk
import spacy
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis.gensim_models
import en_core_web_md
import gensim
import random
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel

nltk.download(['punkt', 
               'stopwords'])

pyLDAvis.enable_notebook()# Visualise inside a notebook

sns.set()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
###########################
# Load CSVs (Github)      #
###########################
workplan_df = pd.read_csv('https://github.com/Colsai/scott_data606/blob/main/Data_Sources/HHS_OIG_workplans.csv?raw=true')
reports_df = pd.read_csv('https://github.com/Colsai/scott_data606/blob/main/Data_Sources/HHS_OIG_Reports.csv?raw=true')

In [5]:
workplan_df.drop(columns = 'Office of Evaluation and Inspections', 
                 inplace = True)

In [6]:
wp_init_srs = [paragraph.lower() for paragraph in workplan_df["Summary"]]

In [7]:
#######################
# Clean Data Here     #
#######################
stopwords = nltk.corpus.stopwords.words('english')

stopwords.append(['on',
                  'or', 
                  'to', 
                  'a', 
                  'as', 
                  'of', 
                  'for',
                  'this', 
                  'by', 
                  's'])

tokenizer = RegexpTokenizer(r'\w+')

tokenized_sums = [[i for i in tokenizer.tokenize(sent) if i not in stopwords] 
                  for sent in wp_init_srs]

# Should we look at more data
'''
ss = nltk.SnowballStemmer(language = 'english')
[[ss.stem(word) for word in sent] for sent in tokenized_sums]
'''

"\nss = nltk.SnowballStemmer(language = 'english')\n[[ss.stem(word) for word in sent] for sent in tokenized_sums]\n"

In [8]:
workplan_df["summary_vector"] = tokenized_sums

In [9]:
workplan_df

,Announced or Revised,Agency,Title,Component,Report Number(s),Expected Issue Date (FY),Website_Link,Summary,summary_vector
0,Completed,Administration for Children and Families,States' Accuracy of Reporting TANF Spending In...,Office of Audit Services,A-02-17-02005; W-00-17-25100,2021,https://oig.hhs.gov/reports-and-publications/w...,The Temporary Assistance for Needy Families (T...,"[temporary, assistance, needy, families, tanf,..."
1,Completed,Administration for Children and Families,Head Start: Review of Single Audit Findings an...,Office of Audit Services,"A-02-16-02009, A-09-16-01004, A-06-17-07003;...",2018,https://oig.hhs.gov/reports-and-publications/w...,Effective for awards made on or after December...,"[effective, awards, made, december, 26, 2014, ..."
2,Completed,Administration for Children and Families,Unaccompanied Children Program Grantee Reviews,Office of Audit Services,A-02-16-02013; A-04-16-03566; A-02-16-02007;...,2020,https://oig.hhs.gov/reports-and-publications/w...,"Under the Homeland Security Act of 2002, § 462...","[homeland, security, act, 2002, 462, office, r..."
3,Nov-16,Administration for Children and Families,Recommendation Follow-Up: Office of Refugee Re...,Office of Evaluation and Inspections,OEI-09-16-00260,2017,https://oig.hhs.gov/reports-and-publications/w...,"Under the Homeland Security Act of 2002, § 462...","[homeland, security, act, 2002, 462, hhs, offi..."
4,October 2020,Administration for Children and Families,ACF Oversight of Guardian Ad Litem Requirement...,Office of Evaluation and Inspections,OEI-12-16-00120,2022,https://oig.hhs.gov/reports-and-publications/w...,As a condition of receiving Child Abuse Preven...,"[condition, receiving, child, abuse, preventio..."
...,...,...,...,...,...,...,...,...,...
649,May 2022,Administration for Children and Families,Audit of the Administration for Children and F...,Office of Audit Services,W-00-22-20035,2023,https://oig.hhs.gov/reports-and-publications/w...,"The Office of Refugee Resettlement (ORR), a pr...","[office, refugee, resettlement, orr, program, ..."
650,May 2022,Centers for Medicare and Medicaid Services,Accuracy of Falls Reporting in Home Health OAS...,Office of Evaluation and Inspections,OEI-05-22-00290,2023,https://oig.hhs.gov/reports-and-publications/w...,"In April 2022, CMS will begin publicly reporti...","[april, 2022, cms, begin, publicly, reporting,..."
651,May 2022,Centers for Medicare and Medicaid Services,Medicare Part B Add-On Payments for COVID-19 T...,Office of Audit Services,W-00-22- 35884,2023,https://oig.hhs.gov/reports-and-publications/w...,Laboratory tests are critical for early detect...,"[laboratory, tests, critical, early, detection..."
652,May 2022,Centers for Medicare and Medicaid Services,Followup Review of Inpatient Claims Under the ...,Office of Audit Services,W-00-22-35885,2023,https://oig.hhs.gov/reports-and-publications/w...,Medicare makes the full Medicare Severity Diag...,"[medicare, makes, full, medicare, severity, di..."


In [10]:
dictionary = Dictionary(workplan_df["summary_vector"])
print(dictionary.token2id)

{'16': 0, '196r': 1, '2015': 2, '5': 3, 'accomplish': 4, 'accuracy': 5, 'acf': 6, 'achieve': 7, 'actual': 8, 'annually': 9, 'assistance': 10, 'award': 11, 'basis': 12, 'billion': 13, 'block': 14, 'cumulative': 15, 'current': 16, 'design': 17, 'designed': 18, 'determine': 19, 'effective': 20, 'expenditures': 21, 'families': 22, 'fiscal': 23, 'form': 24, 'four': 25, 'fourth': 26, 'funds': 27, 'fy': 28, 'grant': 29, 'grants': 30, 'help': 31, 'henceforth': 32, 'information': 33, 'longer': 34, 'made': 35, 'must': 36, 'needy': 37, 'new': 38, 'obligations': 39, 'one': 40, 'open': 41, 'operate': 42, 'period': 43, 'program': 44, 'programs': 45, 'purposes': 46, 'quarter': 47, 'quarterly': 48, 'receive': 49, 'referred': 50, 'reflect': 51, 'reflects': 52, 'report': 53, 'reporting': 54, 'resulting': 55, 'self': 56, 'spending': 57, 'states': 58, 'sufficiency': 59, 'tanf': 60, 'temporary': 61, 'transfers': 62, 'unliquidated': 63, 'using': 64, 'year': 65, '000': 66, '1996': 67, '2013': 68, '2014': 69,

In [11]:
dictionary.filter_extremes(no_below=5, 
                           no_above=0.6, 
                           keep_n=1000)

In [12]:
corpus = [dictionary.doc2bow(doc) for doc in workplan_df["summary_vector"]]

In [13]:
lda_model = LdaMulticore(corpus=corpus, 
                         id2word=dictionary, 
                         iterations=100, 
                         num_topics=8,
                         workers = 8, 
                         passes=20)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

In [24]:
cm = CoherenceModel(model=lda_model, 
                    corpus=corpus, 
                    coherence="u_mass")
coherence = cm.get_coherence()  # get coherence value
print(coherence)

ValueError: ignored

In [89]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.060*"medicare" + 0.024*"part" + 0.022*"drug" + 0.019*"drugs" + 0.017*"cms" + 0.015*"payments" + 0.012*"beneficiaries" + 0.012*"data" + 0.012*"b" + 0.012*"amp"
Topic: 1 
Words: 0.050*"services" + 0.037*"medicare" + 0.021*"payments" + 0.020*"care" + 0.019*"medicaid" + 0.015*"whether" + 0.014*"requirements" + 0.013*"state" + 0.012*"payment" + 0.011*"home"
Topic: 2 
Words: 0.040*"covid" + 0.040*"19" + 0.036*"health" + 0.016*"emergency" + 0.016*"response" + 0.015*"ihs" + 0.014*"pandemic" + 0.014*"facilities" + 0.014*"disease" + 0.014*"care"
Topic: 3 
Words: 0.034*"federal" + 0.027*"hhs" + 0.017*"oig" + 0.016*"program" + 0.014*"funds" + 0.011*"programs" + 0.010*"risk" + 0.010*"requirements" + 0.009*"whether" + 0.009*"report"
Topic: 4 
Words: 0.029*"nih" + 0.025*"nursing" + 0.019*"oig" + 0.016*"health" + 0.014*"research" + 0.013*"data" + 0.013*"cms" + 0.012*"asp" + 0.012*"homes" + 0.011*"home"
Topic: 5 
Words: 0.063*"medicaid" + 0.039*"care" + 0.034*"states" + 0.031*"financ

In [90]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

Topic: 0 Word: 0.008*"home" + 0.007*"children" + 0.006*"providers" + 0.006*"orr" + 0.006*"services" + 0.006*"care" + 0.005*"facilities" + 0.005*"states" + 0.005*"health" + 0.005*"state"
Topic: 1 Word: 0.011*"financial" + 0.007*"nursing" + 0.007*"19" + 0.006*"charge" + 0.006*"hhs" + 0.006*"covid" + 0.006*"audit" + 0.006*"act" + 0.005*"data" + 0.005*"independent"
Topic: 2 Word: 0.011*"data" + 0.009*"opioid" + 0.008*"opioids" + 0.007*"beneficiaries" + 0.007*"medicare" + 0.006*"overdose" + 0.006*"diagnoses" + 0.005*"senior" + 0.005*"order" + 0.005*"risk"
Topic: 3 Word: 0.010*"telehealth" + 0.009*"asp" + 0.007*"drug" + 0.006*"reimbursement" + 0.006*"average" + 0.006*"medicare" + 0.006*"medicaid" + 0.005*"quarters" + 0.005*"services" + 0.005*"states"
Topic: 4 Word: 0.012*"fda" + 0.008*"drug" + 0.008*"opioid" + 0.007*"treatment" + 0.007*"drugs" + 0.006*"medicare" + 0.006*"services" + 0.006*"food" + 0.006*"medicaid" + 0.005*"part"
Topic: 5 Word: 0.010*"medicare" + 0.008*"inpatient" + 0.008*"ho

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [91]:
#corpus = random.randint(0,len(corpus))

for index, score in sorted(lda_model_tfidf[corpus[450]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.4929039478302002	 
Topic: 0.008*"home" + 0.007*"children" + 0.006*"providers" + 0.006*"orr" + 0.006*"services" + 0.006*"care" + 0.005*"facilities" + 0.005*"states" + 0.005*"health" + 0.005*"state"

Score: 0.44950801134109497	 
Topic: 0.007*"children" + 0.006*"drugs" + 0.006*"response" + 0.006*"covid" + 0.006*"19" + 0.005*"hhs" + 0.005*"state" + 0.004*"part" + 0.004*"cms" + 0.004*"surveys"

Score: 0.04954013228416443	 
Topic: 0.011*"data" + 0.009*"opioid" + 0.008*"opioids" + 0.007*"beneficiaries" + 0.007*"medicare" + 0.006*"overdose" + 0.006*"diagnoses" + 0.005*"senior" + 0.005*"order" + 0.005*"risk"


In [101]:
import pyLDAvis
import pyLDAvis.gensim_models
# feed the LDA model into the pyLDAvis instance
vis = pyLDAvis.gensim_models.prepare(ldamodel, 
                                     doc_term_matrix, 
                                     dictionary)

pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

NameError: ignored

In [93]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.07495936568097064),
 (1, 0.06234273443933126),
 (2, 0.08519101190277396),
 (3, 0.2812932644833637),
 (4, 0.10032330994235517),
 (5, 0.0910146873317962),
 (6, 0.07426336880204319),
 (7, 0.17038202380554793),
 (8, 0.07426336880204319),
 (9, 0.06278137226415441),
 (10, 0.11185440966730993),
 (11, 0.08519101190277396),
 (12, 0.09525616099703547),
 (13, 0.06558030217413534),
 (14, 0.5175383882472531),
 (15, 0.13316841620914108),
 (16, 0.23873088837401302),
 (17, 0.08852725021343395),
 (18, 0.05118715682227938),
 (19, 0.07794528549382433),
 (20, 0.12382269431883122),
 (21, 0.07165135704989023),
 (22, 0.07957696279133766),
 (23, 0.04773769344801265),
 (24, 0.10294477475934799),
 (25, 0.04700248042913429),
 (26, 0.0636814040824336),
 (27, 0.11185440966730993),
 (28, 0.069850757892577),
 (29, 0.11185440966730993),
 (30, 0.09525616099703547),
 (31, 0.07228026305324124),
 (32, 0.057583890908616736),
 (33, 0.04823959999770438),
 (34, 0.1066172383862893),
 (35, 0.21820935100930308),
 (36, 0.